<a href="https://colab.research.google.com/github/sugarforever/Advanced-RAG/blob/main/03_llama_index_multi_doc_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 查询路由和Multi-Document Agent相关概念



# 查询路由
查询路由是LLM驱动的关键决策环节，它决定了在接收到用户查询后应采取的最佳行动——可能的行动包括生成摘要、对特定数据索引执行搜索或尝试多种不同的路由策略，并将它们的输出汇总成一个统一的答案。

查询路由还负责选择最合适的数据存储位置来响应用户查询。这些数据存储位置可以是多种形式，如传统的向量存储、图形数据库、关系型数据库，或是分层次的索引系统。在处理包含多个文档的存储时，通常会利用两种不同的索引方式：摘要索引和文档块向量索引。

定义查询路由器涉及设定它可以采取的行动选项。

选择特定路由的过程是通过调用大型语言模型来实现的，其结果会按照预定义的格式返回，以便将查询定向到指定的索引。如果涉及到复杂的关联操作，这些查询还可能被发送到子链或其他智能体，如下文所述的多文档智能体方案所示。

LlamaIndex和LangChain都提供了对查询路由的支持。

## 智能体（Agent）
在智能体技术的实现上，尤其是在基于大型语言模型（LLM）的智能体构建中，LLM在智能体的智能化中扮演着至关重要的角色。这些智能体能够通过整合LLM与规划、记忆以及其他关键技术模块，执行复杂的任务。在此框架中，LLM充当核心处理单元或“大脑”，负责管理和执行为特定任务或响应用户查询所需的一系列操作。
智能体也是Langchain和LlamaIndex支持的一个概念，它从第一个LLM API发布之初就已存在。这个概念旨在为具备推理能力的LLM提供一套工具和任务。这些工具可能包括一些确定性的功能，如代码函数或外部API，甚至包括其他智能体——这种LLM之间的链接是LangChain名称的灵感来源。
智能体本身是一个复杂的技术，下面将继续基于智能体的多文档检索案例，并在下文将要介绍的RAG系统中发挥作用。
在LlamaIndex中，有一个OpenAIAgent类，它将这种高级逻辑与ChatEngine和QueryEngine类结合在一起，提供基于知识和上下文感知的聊天，以及在一个对话轮次中调用多个OpenAI函数的能力，这真正实现了智能代理行为。

## 多文档智能体方案
![mda](.\mda.png)

多文档智能体的方案是一个高度复杂的配置，它在每个文档上初始化一个Agent（OpenAIAgent），该智能体能进行文档摘要制作和传统问答机制的操作，还有一个顶层智能体，负责将查询分配到各个文档智能体，并综合形成最终的答案。

每个文档智能体都配备了两个工具：向量存储索引和摘要索引，它根据路由查询来决定使用哪一个。对于顶层智能体来说，所有文档智能体都是其工具。

该方案展示了一种高级RAG架构，其中每个智能体都进行多个决策的导航。这种方法的优势在于能够比较不同的解决方案或实体在不同文档及其摘要中的描述，以及传统的单个文档摘要和QA机制——这基本上涵盖了与文档集合聊天的最常见用例。

这种复杂配置的缺点在于，由于需要在智能体内部的大语言模型之间进行多次往返迭代，其运行速度较慢。值得一提的是，LLM调用通常是RAG管道中最耗时的操作，而搜索则是为了速度而优化的设计。因此，对于大型的多文档存储，建议考虑对这种方案进行简化，以便实现扩展。

下面将根据llamaindex官方示例进行讲解

In [1]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-你的sk"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [32]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

In [33]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
]

## 将爬取到的文档存在本地 将在下面构建多文档智能体

In [35]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w",encoding="utf-8") as fp:
        fp.write(wiki_text)

In [36]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

In [37]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")


## 在下面小节中，将展示如何构建多文档智能体。首先为每个文档构建一个文档智能体，然后定义一个带有对象索引的顶层父级智能体。

为每个文档构建文档智能体
1.在本节中，为每个文档定义了“文档智能体”。

2.为每个文档定义了一个向量索引（用于语义搜索）和摘要索引（用于生成摘要）。然后将这两个查询引擎转换为工具，并传递给调用OpenAI函数的智能体。

3.该文档智能体可以动态选择在给定文档内执行语义搜索或生成摘要。

4.我们为每个城市文档创建了一个单独的文档智能体。

In [39]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os

node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])
    all_nodes.extend(nodes)

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    summary_query_engine = summary_index.as_query_engine(llm=llm)

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for any requests that require a holistic summary"
                    f" of EVERYTHING about {wiki_title}. For questions about"
                    " more specific sections, please use the vector_tool."
                ),
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-3.5-turbo")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {wiki_title}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    agents[wiki_title] = agent
    query_engines[wiki_title] = vector_index.as_query_engine(
        similarity_top_k=2
    )

# 构建检索器支持的OpenAI智能体

下面构建了一个顶层智能体，它可以协调不同的文档智能体来回答用户的任何查询。

这个智能体将所有文档智能体作为工具。这个特定的智能体`RetrieverOpenAIAgent`在工具使用前执行工具检索（不同于默认智能体，后者尝试将所有工具放入提示中）。

在这里，使用了一个top-k检索器，也可以自定义其他工具检索方法！

In [40]:
# define tool for each document agent
all_tools = []
for wiki_title in wiki_titles:
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        f" this tool if you want to answer any questions about {wiki_title}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=agents[wiki_title],
        metadata=ToolMetadata(
            name=f"tool_{wiki_title}",
            description=wiki_summary,
        ),
    )
    all_tools.append(doc_tool)

In [41]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [42]:
from llama_index.agent.openai import OpenAIAgent

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an agent designed to answer queries about a set of given cities.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)

# 定义基线向量存储索引

作为比较的基准，我们定义了一个“朴素”的RAG流程，它将所有文档倾泻到一个单一的向量索引集合中。

我们设置 top_k = 4

In [43]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

# 执行示例查询

执行一些示例查询，范围从单个文档的问答/摘要到多个文档的问答/摘要。

In [44]:
# should use Boston agent -> vector tool
response = top_agent.query("Tell me about the arts and culture in Boston")

Added user message to memory: Tell me about the arts and culture in Boston
=== Calling Function ===
Calling function: tool_Boston with args: {"input":"arts and culture"}
Added user message to memory: arts and culture
=== Calling Function ===
Calling function: vector_tool with args: {"input":"arts and culture in Boston"}
Got output: Boston has a rich cultural scene with a strong emphasis on arts and music. The city is home to several art museums and galleries, including the Museum of Fine Arts and the Isabella Stewart Gardner Museum. The Boston Symphony Orchestra, one of the "Big Five" American orchestras, is based in Symphony Hall. Boston also hosts various performing arts organizations, such as the Boston Ballet, Boston Lyric Opera Company, and the Handel and Haydn Society. Additionally, the city is known for its literary culture, with historical figures like Ralph Waldo Emerson and Nathaniel Hawthorne contributing to its reputation as "the intellectual capital of the United States."


In [45]:
print(response)

Boston has a rich cultural scene with a strong emphasis on arts and music. The city is home to several art museums and galleries, including the Museum of Fine Arts and the Isabella Stewart Gardner Museum. The Boston Symphony Orchestra, one of the "Big Five" American orchestras, is based in Symphony Hall. Boston also hosts various performing arts organizations, such as the Boston Ballet, Boston Lyric Opera Company, and the Handel and Haydn Society. Additionally, the city is known for its literary culture, with historical figures like Ralph Waldo Emerson and Nathaniel Hawthorne contributing to its reputation as "the intellectual capital of the United States."


In [46]:

# baseline
response = base_query_engine.query(
    "Tell me about the arts and culture in Boston"
)
print(str(response))

Boston has a rich arts and culture scene, with numerous art museums and galleries like the Museum of Fine Arts, Isabella Stewart Gardner Museum, and Institute of Contemporary Art. The city also boasts a vibrant music scene, highlighted by the Boston Symphony Orchestra, one of the "Big Five" American orchestras. Annual events such as the Boston Early Music Festival, Boston Arts Festival, and the Boston gay pride parade contribute to the city's cultural vibrancy. Boston also has a strong literary history, with renowned writers like Ralph Waldo Emerson, Nathaniel Hawthorne, and Henry Wadsworth Longfellow shaping the city's intellectual landscape.


In [16]:

# should use Houston agent -> vector tool
response = top_agent.query(
    "Give me a summary of all the positive aspects of Houston"
)

In terms of history, the Premier League was established in 1992 and has since become one of the most popular and financially lucrative football leagues in the world. It is known for its competitive nature and has been dominated by clubs like Manchester United, Arsenal, Chelsea, Liverpool, and Manchester City. The league has also seen remarkable moments, such as Leicester City's title victory in the 2015-2016 season.

On the other hand, La Liga has a longer history, dating back to 1929. It has been home to some of the most successful and iconic clubs in football, including Real Madrid and Barcelona. The league has witnessed periods of dominance from various clubs and has seen intense rivalries between teams. Real Madrid has been particularly successful in the Champions League, winning numerous titles.

In terms of UEFA Champions League (UCL) performance, both leagues have had strong showings. English clubs have won a total of 15 UCL titles, making England the second-most successful coun

In [47]:
# baseline
response = base_query_engine.query(
    "Give me a summary of all the positive aspects of Houston"
)
print(str(response))

Houston is a vibrant city with a robust economy fueled by exports, especially in petroleum products and oil-related industries. It has a diverse and expanding international community, showcasing various cultural events that highlight its multicultural essence. The city is famous for its culinary scene and restaurant culture, along with a lively arts and theater scene featuring numerous performing arts organizations and museums. Houston is a sports hub, hosting teams in major professional leagues, and offers a plethora of tourist attractions, parks, and recreational areas, making it an attractive destination for residents and visitors alike.


In [48]:
# baseline: the response doesn't quite match the sources...
response.source_nodes[1].get_content()

'The city is home to the nation\'s third-largest concentration of consular offices, representing 92 countries.Many annual events celebrate the diverse cultures of Houston. The largest and longest-running is the annual Houston Livestock Show and Rodeo, held over 20 days from early to late March, and is the largest annual livestock show and rodeo in the world. Another large celebration is the annual night-time Houston Gay Pride Parade, held at the end of June. Other notable annual events include the Houston Greek Festival, Art Car Parade, the Houston Auto Show, the Houston International Festival, and the Bayou City Art Festival, which is considered to be one of the top five art festivals in the United States.Houston is highly regarded for its diverse food and restaurant culture. Houston received the official nickname of "Space City" in 1967 because it is the location of NASA\'s Lyndon B. Johnson Space Center. Other nicknames often used by locals include "Bayou City", "Clutch City", "Crus

In [49]:

response = top_agent.query(
    "Tell the demographics of Houston, and then compare that with the"
    " demographics of Chicago"
)

Added user message to memory: Tell the demographics of Houston, and then compare that with the demographics of Chicago
=== Calling Function ===
Calling function: tool_Houston with args: {"input": "demographics"}
Added user message to memory: demographics
=== Calling Function ===
Calling function: summary_tool with args: {"input":"demographics"}
Got output: Houston is known for its diverse population, with various ethnic and religious backgrounds represented. The city has been described as the most racially and ethnically diverse major city in the U.S. Houston's population includes a large and growing international community, contributing to its status as a global city. Additionally, the city has a significant African American population, with historical ties to the post-Civil War era and the Great Migration.

Got output: Houston is known for its diverse population, with various ethnic and religious backgrounds represented. The city has been described as the most racially and ethnically

In [50]:
# baseline
response = base_query_engine.query(
    "Tell me the differences between Shanghai and Beijing in terms of history"
    " and current economy"
)
print(str(response))

Shanghai's history is marked by international trade and foreign influence, especially during the 19th and early 20th centuries, shaping its identity as a major commercial and financial center. In contrast, Beijing has a more traditional background as the political and cultural heart of China, emphasizing governance and imperial legacy.

Currently, Shanghai is renowned for its emphasis on finance, innovation, and global trade, making a significant contribution to China's GDP. On the other hand, Beijing is known for its role as the political capital, housing government institutions and headquarters of state-owned enterprises, focusing on administrative functions and policy-making.


参考链接：https://docs.llamaindex.ai/en/v0.10.33/